<a href="https://colab.research.google.com/github/pranesh008/Resume-extract/blob/main/resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pdfplumber
import json
import re


In [3]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 66.7 MB/s eta 0:00:00


In [6]:

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open("/content/Pranesh Puri - resume .pdf") as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text


In [9]:
def parse_resume(text):
    # Extract contact information
    contact_info = {
        "name": re.search(r'^(.+?)\n', text).group(1).strip(),
        "email": re.search(r'(\S+@\S+)', text).group(1).strip(),
        "phone": re.search(r'(\+\d+ \d+)', text).group(1).strip(),
        "location": re.search(r'(\w+\, \w+)', text).group(1).strip(),
       # "linkedin": re.search(r'linkedin:(\S+)', text, re.IGNORECASE).group(1).strip()
    }

    # Extract profile
    profile = re.search(r'PROFILE\n(.+?)\nEDUCATION', text, re.DOTALL).group(1).strip()

    # Extract education
    education_sections = re.findall(r'([A-Z][\w\s]+)\n(.+?)\n(CGPA|Percentage|Year|[\d]{4} – [\d]{4}|[A-Za-z\s]+)\n*(.+?)\n*(Pune|[\d]{4})', text, re.DOTALL)
    education = []
    for section in education_sections:
        education.append({
            "degree": section[0].strip(),
            "institution": section[1].strip(),
            "details": section[2].strip(),
            "year": section[3].strip() if len(section) > 3 else None,
            "location": section[4].strip() if len(section) > 4 else "Pune"
        })

    # Extract projects
    projects = re.findall(r'([^\n]+)\n([^\n]+)\n• (.+?)\n• (.+?)\n', text, re.DOTALL)
    project_list = []
    for project in projects:
        project_list.append({
            "title": project[0].strip(),
            "technologies": [tech.strip() for tech in project[1].split(',')],
            "description": project[2].strip() + ' ' + project[3].strip()
        })

    # Extract technical skills
    technical_skills = {
        "fundamentals": re.search(r'Fundamental skills : (.+)', text).group(1).strip().split(','),
        "programming_languages": re.search(r'Programming languages : (.+)', text).group(1).strip().split(','),
        "database": re.search(r'Database : (.+)', text).group(1).strip(),
        "course_work": re.search(r'Course Work : (.+)', text).group(1).strip().split(','),
        "cloud_databases": re.search(r'Cloud databases : (.+)', text).group(1).strip().split(','),
        "text_analysis": re.search(r'Text analysis : (.+)', text).group(1).strip().split(','),
        "developer_tools": re.search(r'Developer Tools : (.+)', text).group(1).strip().split(',')
    }

    # Extract soft skills
    soft_skills = re.search(r'SOFT SKILLS\n([\s\S]+?)\nEXTRA - CURRICULAR', text).group(1).strip().split('\n• ')

    # Extract extra-curricular activities
    extra_curricular = re.findall(r'([^\n]+)\n([^\n]+)\n• (.+?)\n(\d{4} – \d{4})\n• (.+)', text, re.DOTALL)
    extra_curricular_list = []
    for activity in extra_curricular:
        extra_curricular_list.append({
            "role": activity[0].strip(),
            "organization": activity[1].strip(),
            "responsibilities": [resp.strip() for resp in activity[2].split('\n• ')],
            "years": activity[3].strip()
        })

    resume_data = {
        "contact_info": contact_info,
        "profile": profile,
        "education": education,
        "projects": project_list,
        "technical_skills": technical_skills,
        "soft_skills": soft_skills,
        "extra_curricular": extra_curricular_list
    }

    return resume_data

def main():
    file_path = 'resume.txt'  # Change this to your resume file path
    resume_text = extract_text_from_pdf(file_path)
    parsed_data = parse_resume(resume_text)
    json_data = json.dumps(parsed_data, indent=4)
    with open('resume_data.json', 'w') as f:
        f.write(json_data)
    print("Resume parsed and saved to 'resume_data.json'")

if __name__ == "__main__":
    main()


Resume parsed and saved to 'resume_data.json'
